# SARAH Toolbox
## Collection of models to be run from a docker image
Launch virtual docker image: 
[![Binder](http://mybinder.org/badge.svg)](http://mybinder.org/repo/restrepo/SimplifiedDM-SDFDM-Toolbox)

[Jupyter home](./) (Files, New -> Terminal, etc)
[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./)

### Highly recommended:
[Run from a terminal](../terminals/1):
[![T](http://mysitemyway.com/_shared/images/icons/full_color/terminal.png)](../terminals/1)
## Implemented models
Each model is to be run in a specific virtual machine. Follow the binder button in each github repo
* `SARAH/Models/SSDM/` (for test purposes)
* `SARAH/Models/SM/HighScale` (For RGE running: ./butler SM/HighScale) [Repo](https://github.com/restrepo/SM-Toolbox/)
* `SARAH/Models/SimplifiedDM/DFDM` (This image)
* `SARAH/Models/SimplifiedDM/SDFDM` (This image)

## Instructions to compile the model
In `SPHENO` and `micromegas`

See possible analysis based on the models in  [./tests](./tests) folder.

Below we define the model to be compiled:

In [1]:
MODEL=ZpBminusLSM

To better control in the outputs, it is recomended that the commands be executed [from a terminal](../terminals/1). Only possible errors are to be shown below

In [58]:
./compile_spheno_directly.sh $MODEL > /dev/null

make a work dir

In [2]:
mkdir -p test_compilation
cd test_compilation

In [3]:
LHAINPUT=../SPHENO/$MODEL/Input_Files/LesHouches.in.$MODEL
if [ -f ../SARAH/Models/$MODEL/LesHouches.in.$MODEL ]; then
   LHAINPUT=../SARAH/Models/$MODEL/LesHouches.in.$MODEL
fi
cp $LHAINPUT . #only a number of limited charactes accepted as input

Run SPheno and generate LHA output file. However, there is one caveat in `MadGraph`: `MadEvent` has problems with reading the HiggsBounds specific blocks in the SPheno spectrum file (`HiggsBoundsInputHiggsCouplingsFermions` and `HiggsBoundsInputHiggsCouplingsBosons`). If these blocks are included, MadEvent won’t accept the file. Therefore, The `HiggsBounds` blocks need to be disabled:

In [8]:
sed -ri 's/(^520) 1/\1 0/' LesHouches.in.$MODEL

In fact:

In [9]:
grep ^520 LesHouches.in.$MODEL

520 0.              # Write effective Higgs couplings (HiggsBounds blocks): put 0 to use file with MadGraph! 


We now run the spectrum generator

In [7]:
../SPHENO/bin/SPheno$MODEL LesHouches.in.$MODEL

 Calculating branching ratios and decay widths
 Calculating low energy constraints
 Writing output files
 Finished!


with the LHA output:

In [10]:
ls SPheno.spc.$MODEL

SPheno.spc.BLSM


##  Madgraph

We are using [MG5_aMC_v2_3_3](https://launchpad.net/mg5amcnlo/2.0/2.3.x/+download/MG5_aMC_v2.3.3.tar.gz).

We follow here clossely the SARAH Manual: _Exploring new models in all detail with_ `SARAH`, [arxiv.org:1503.04200](https://arxiv.org/pdf/1503.04200.pdf), Sec. "Dilepton analysis with MadGraph", pag. 118, but adapted to our problem.

The command to generate the UFO files is

 `MakeUFO []`
 
Within the SARAH toolbox used here, this `Mathematica` command is automatically executed and the resulting code is copied into the proper `MadGraph` folder at `madgraph/models/BLSM`

Here, we use an input file for MadGraph: `install_and_check.mdg`  which contains the following lines.
(If all tests are passed hit `<ENTER>`)

In [28]:
cat << EOF > install_and_check.mdg #Creates the file with the folloing content before EOF
install pythia-pgs
# Check your root installation and install Delphes
install Delphes
# Check SARAH model
import model BLSM -modelname
# p is already defined if imported from	MadGraph console
define p g d1 d1bar d2 d2bar u1 u1bar u2 u2bar
check p p > nu4 nu4
EOF

In [30]:
cd ../madgraph  
./bin/mg5_aMC ../test_compilation/install_and_check.mdg 2>/dev/null | tail -n20
cd ../test_compilation

Gauge results (switching between Unitary/Feynman):
Process          Unitary           Feynman           Relative diff.    Result
Summary: 0/0 passed, 0/0 failed
Process permutation results:
Process            Min element       Max element       Relative diff.    Result
g g > nu4 nu4      0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  Passed
d1 d1bar > nu4 nu4 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  Passed
d1 d2bar > nu4 nu4 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  Passed
d2 d1bar > nu4 nu4 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  Passed
d2 d2bar > nu4 nu4 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  Passed
u1 u1bar > nu4 nu4 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  Passed
u1 u2bar > nu4 nu4 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  Passed
u2 u1bar > nu4 nu4 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  Passed
u2 u2bar > nu4 nu4 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  Passed
Summary: 9

Now we can generate some processes. We just check that the lightest righ handed neutrino does no couple with the photon.
The possible VBF processes is in a commented line.

In [41]:
cat << EOF > VBF.mdg #Creates the file with the folloing content before EOF 
#See: http://www.niu.edu/spmartin/madgraph/madsyntax.html
import model BLSM -modelname
#p is already defined if imported from	MadGraph console
define p g d1 d1bar d2 d2bar u1 u1bar u2 u2bar
define j g d1 d1bar d2 d2bar u1 u1bar u2 u2bar   
#generate p p > nu4 nu4 j j \$\$ z zp wm wmc h1 h2 a QED=99 QCD=99 @1
##Only VBF Zp
#generate p p > nu4 nu4 j j \$\$ z zp wm wmc h1 h2 a / z wm wmc h1 h2 a nu5 nu6  QED=99 QCD=99 @1
##Only drell Yann Zp
#generate p p > e1 e1bar \$\$ z zp wm wmc h1 h2 a / z wm wmc h1 h2 a  QED=99 QCD=99 @1
generate p p > nu4 nu4 j j \$\$ z zp wm wmc h1 h2 a QED=99 QCD=99 @1
#Just check that right-handed neutrino does no couples to foton:
generate p p > nu4 nu4 \$\$ z zp wm wmc h1 h2 QED=99 QCD=99 @1
output ../test_compilation/MadOut
exit
EOF
../madgraph/bin/mg5_aMC VBF.mdg  2> /dev/null | tail -n15

save configuration file to /home/restrepo/prog/toolbox/toolbox/test_compilation/MadOut/Cards/me5_configuration.txt
INFO: Use Fortran compiler gfortran 
INFO: Generate jpeg diagrams 
INFO: Generate web pages 
Output to directory /home/restrepo/prog/toolbox/toolbox/test_compilation/MadOut done.
Type "launch" to generate events from this process, or see
/home/restrepo/prog/toolbox/toolbox/test_compilation/MadOut/README
Run "open index.html" to see more information about this process.
exit
 
quit
 

 



In the first of line `VBF.mdg`, we import the model in `MadGraph`. The option `-modelname` is used to keep the names of the particles as given in the model files. By default, `MadGraph` will try to use the default naming conventions. However, this would fail for this model. We define a multi-particle called `p` which consists of gluon and all the light quarks. If we import the model direcly from the `MadGraph` console, `p` is definded automatically from the new model name particles.

The `n4` is the lightest right handed neutrino. Thus, in the third line we generate the process $p p \to N_R\;N_R\;j\;j$ in mediated by the phothon which must be rise to a zero cross section. The output for MadEvent is written to a new subdirectory `test_compilation/MadOut`  and we close `MadGraph` when it is done via exit. 

After MadGraph has created the output for MadEvent and finished, we can enter the new subdirectory `test_compilation/MadOut`. The important settings to generate events are done via the files in the Cards-directory: the file `param_card.dat` is used to give the input for all parameters, `run_card.dat` controls the event generations. In the last file, the user can for instance set the beam type and energy, define the renormalization scale, apply cuts, and fix the number of events. We want to use, of course, the spectrum file as written by SPheno. 

When we have the spectrum file in the correct form, we can copy this file to the Cards directory as
`param_card.dat`.

In [39]:
cp SPheno.spc.$MODEL MadOut/Cards/param_card.dat 

Changes in any of the input parameters, require to update the `SPheno` output and overwrite the `MadOut/Cards/param_card.dat` file

In [42]:
cat << EOF > launch.mdg
launch MadOut 
0
0
EOF
../madgraph/bin/mg5_aMC launch.mdg

************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 2.3.3                 2015-10-25         *
*                                                          *
*    The MadGraph5_aMC@NLO Development Team - Find us at   *
*    https://server06.fynu.ucl.ac.be/projects/madgraph     *
*                       

## Go now to: 
* [main ipyrhon file](./tests/BLSM.ipynb) → $B-L$ extended standard model